## Preparando Pytorch

In [1]:

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os
import cv2
import numpy as np

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Dataset class
class CovidDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        self.image_dir = image_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []


# Define transformations
data_transforms = {
    'raw': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Load pre-trained models
model_resnet = models.resnet18(pretrained=True).to(device)
num_ftrs = model_resnet.fc.in_features
model_resnet.fc = nn.Linear(num_ftrs, 3).to(device)  # 3 output classes

model_vgg = models.vgg16(pretrained=True).to(device)
model_vgg.classifier[6] = nn.Linear(4096, 3).to(device) # 3 output classes

# Freeze the pre-trained layers
for param in model_resnet.parameters():
    param.requires_grad = False
for param in model_vgg.parameters():
    param.requires_grad = False

c:\Users\Administrator\Documents\Projects\Covid\.venv\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Administrator\Documents\Projects\Covid\.venv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\Administrator/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:01<00:00, 25.0MB/s]
c:\Users\Administrator\Documents\Projects\Covid\.venv\lib\site-packages\torchvision\models\_utils.py:223: User

## Preparando el Dataset

In [2]:
class CovidDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        self.image_dir = image_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        self.class_names = ['covid', 'normal', 'pneumonia']

        # Iterate through each class folder and append file paths and labels
        for idx, class_name in enumerate(self.class_names):
            class_dir = os.path.join(image_dir, class_name)
            for img_file in os.listdir(class_dir):
                self.image_paths.append(os.path.join(class_dir, img_file))
                self.labels.append(idx)  # Assign label based on class index

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]

        # Read image using PIL
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image, label


# Resnet

In [5]:
# Define the training function
def train_model(model, criterion, optimizer, dataloaders, dataset_sizes, num_epochs=25):
    model.train()  # Set model to training mode

    for epoch in range(num_epochs):
        running_loss = 0.0
        running_corrects = 0

        # Iterate over data.
        for inputs, labels in dataloaders['train']:
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            # Backward pass + optimize
            loss.backward()
            optimizer.step()

            # Statistics
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / dataset_sizes['train']
        epoch_acc = running_corrects.double() / dataset_sizes['train']

        print(f'Epoch {epoch}/{num_epochs - 1}, Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}')

    return model

# Create dataset and dataloaders
image_dir = 'dataset/2/Covid19-dataset/test'
train_dataset = CovidDataset(image_dir=image_dir, transform=data_transforms['raw'])
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)

dataloaders = {'train': train_loader}
dataset_sizes = {'train': len(train_dataset)}

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer_resnet = optim.SGD(model_resnet.fc.parameters(), lr=0.001, momentum=0.9)

# Train the model
model_resnet = train_model(model_resnet, criterion, optimizer_resnet, dataloaders, dataset_sizes, num_epochs=25)

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'dataset/2/Covid19-dataset/test\\pneumonia'

## VGG